<a href="https://colab.research.google.com/github/aslla77/practice-Python/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8/06_%EA%B2%80%EC%A6%9D_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0_%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#검증 세트(Validation set)


In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
wine=pd.read_csv('http://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
data=wine.values[:,:-1]
data[:5]

array([[9.4 , 1.9 , 3.51],
       [9.8 , 2.6 , 3.2 ],
       [9.8 , 2.3 , 3.26],
       [9.8 , 1.9 , 3.16],
       [9.4 , 1.9 , 3.51]])

In [4]:
target=wine['class'].to_numpy()
print(target[:5])

[0. 0. 0. 0. 0.]


In [5]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2, random_state=42)

In [6]:
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [7]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [8]:
print(sub_input.shape[0]/len(data))
print(val_input.shape[0]/len(data))
print(test_input.shape[0]/len(data))

0.6398337694320455
0.1600738802524242
0.20009235031553024


In [9]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


#교차검증
- 검증 세트를 만드느라 훈련 세트가 줄었습니다.
- 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용 가능

# k-폴드 교차 검증
- k는 보통 3, 5, 10- 폴드 교차 검증을 많이 사용합니다. 이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다.


In [10]:
from sklearn.model_selection import cross_validate
scores=cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01112342, 0.00757623, 0.00766492, 0.0074079 , 0.00731874]), 'score_time': array([0.00106335, 0.00081658, 0.00085998, 0.00079489, 0.00099111]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [11]:
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
from sklearn.model_selection import StratifiedKFold

scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


#하이퍼 파라미터 튜닝

- 모델 파라미터: 머신러닝 모델이 학습하는 파라미터
- 하이퍼 파라미터: 모델이 학습할 수 없어서 사용자가 지정 해줘야만 하는 파라미터

파이썬의 for 문을 사용하여 찾을 수도 있지만 싸이킷런이 제공하는 그리드 서치를 사용합시다.

그리드 서치는 하이퍼파라미터 탐색과 교차검증을 한번에 수행합니다.


In [22]:
from sklearn.model_selection import GridSearchCV
def mul(x):
  return round(x*(0.0001),4)
params={'min_impurity_decrease':list(map(mul,[1,2,3,4,5]))}
print(params)

{'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}


In [23]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [24]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [25]:
dt=gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [26]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [27]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [28]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [29]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}

#9*15*10=1350개 5-fold cross validation일 경우 1350*5= 6750개

In [30]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [31]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [32]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


#랜덤서치

- 매개변수의 값이 수치일 때
- 값의 범위나 간격을 미리 정하기 어려울 수 있습니다.
- 또 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있습니다, 이럴 때 랜덤 서치를 사용합니다


In [33]:
from scipy.stats import uniform, randint

In [35]:
rgen=randint(0,10)
rgen.rvs(10)

array([0, 1, 2, 5, 3, 7, 8, 0, 9, 1])

In [36]:
#더 많은 숫자의 샘플을 골라 각 숫자의 개수를 새어 분포를 확인
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([111, 109,  96, 103, 101, 108,  95,  89,  86, 102]))

In [37]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.11101831, 0.22116206, 0.3692314 , 0.99717466, 0.93468219,
       0.07642555, 0.68389769, 0.9508691 , 0.80997403, 0.43970647])

In [38]:
#난수 발생기랑 유사하게 하는 클래스로 매개변수의 딕셔너리를 만든다
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25),
        }
        

In [39]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                      n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe7f1ea4e90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe7f1ea4dd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe7f306b7d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe7f1ea4310>},
                   random_state=42)

In [40]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [42]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [43]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
